# Submission 1: CatBoost (plain tuned model)

In [1]:
# fundamentals
import matplotlib.pyplot as plt
import seaborn as sns 
import pandas as pd 
import numpy as np
import scipy
import math

# data exploration 
from pandas_profiling import ProfileReport
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=True, world_readable=True)
from plotly.offline import iplot

# data preprocessing 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, PowerTransformer, MinMaxScaler
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import warnings
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm


# regressors
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR, SVR

import xgboost as xgb 
import catboost as cb
import optuna.integration.lightgbm as lgb

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

# metrics for evaluation
from sklearn.metrics import mean_squared_error
from scipy import stats

# saving parameters
from joblib import dump, load

# hyperparameter searching and tuning 
import optuna
import tqdm

In [2]:
#fixing random seed for repoducability
import random

random.seed(0)
np.random.seed(0)

In [3]:
original_df = pd.read_csv('train.csv', index_col = 'id')

In [4]:
train, val = train_test_split(original_df, random_state = 44, train_size = 0.9)

In [5]:
categorical_col = ['cat0','cat1','cat2','cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']

In [6]:
X_train = train.drop(columns = 'target', axis =1)
Y_train = train['target']
X_val = val.drop(columns = 'target', axis =1)
Y_val = val['target']

In [7]:
X_train.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
id,,,,,,,,,,,,,,,,,,,,,
177066,A,B,A,A,B,B,A,E,E,M,...,0.321770,0.942681,0.237631,0.690661,0.320834,0.416843,0.437228,0.484102,0.268153,0.380358
144442,A,A,A,A,B,D,A,E,G,A,...,0.279534,0.393689,0.472925,0.340930,0.438995,0.926913,0.657195,0.610818,0.336289,0.296117
330061,A,A,A,C,B,B,A,E,E,F,...,0.726732,0.168633,0.326138,0.271090,0.393662,0.286381,0.225950,0.257859,0.325431,0.772950
374221,A,A,A,C,B,B,A,E,E,L,...,0.809864,0.235436,0.366366,0.393269,0.381500,0.315393,0.333075,0.366121,0.344163,0.263223
34764,A,B,A,C,B,B,A,E,A,F,...,0.301346,0.420276,0.640848,0.281902,0.429841,0.390938,0.692147,0.335246,0.340002,0.729724


In [48]:
X_train.columns

Index(['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8',
       'cat9', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6',
       'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13'],
      dtype='object')

In [86]:
class KFoldsAverageCBTuning():
    def __init__(self):
        self.models = []
        self.kfolds = KFold(n_splits = 5, shuffle = False)
        
        
    def fit(self, train_x, train_y, params):
        oof_preds = np.zeros_like(train_y)
        self.train_x = train_x.values
        self.train_y = train_y.values
        
        for train_idx, val_idx in self.kfolds.split(train_x):
            X_trainCV, X_valCV = self.train_x[train_idx], self.train_x[val_idx]
            Y_trainCV, Y_valCV = self.train_y[train_idx], self.train_y[val_idx]
            
            model = cb.CatBoostRegressor(
                        random_state = 50,
                        loss_function = 'RMSE',
                        silent = True,
                        learning_rate = 1,
                        iterations = 20000,
                        cat_features = list(range(10)),
                        **params
                        )
            
            model.fit(X_trainCV, Y_trainCV, 
                      early_stopping_rounds=500,
                      eval_set=[(X_valCV, Y_valCV)], verbose_eval=1000)
            
            self.models.append(model)
            oof_pred = model.predict(X_valCV)
            oof_preds[val_idx] = oof_pred
            
        self.oof_preds = oof_preds
        
        self.rmse = mean_squared_error(Y_train, oof_preds, squared = False)

    def predict(self, test_x):
        preds = []
        for model in tqdm(self.models):
            pred = model.predict(test_x)
            preds.append(pred)
        preds = np.mean(preds, axis=0)
        return preds

In [87]:
def objective_CB(trial):
    l2_leaf_reg = trial.suggest_loguniform('l2_leaf_reg',1e-10, 1)
    min_child_samples = trial.suggest_int('min_child_samples', 1, 41, 2)
    max_depth = trial.suggest_int('max_depth', 2, 16, step = 1)
    subsample = trial.suggest_float('subsample', 0, 1, step = 0.00001)
    colsample_bylevel = trial.suggest_loguniform('colsample_bylevel',1e-10, 1)
    bagging_temperature = trial.suggest_int('bagging_temperature', 0,200000, step = 1)
    random_strength = trial.suggest_int('random_strength', 1, 40, step = 1)
    
    objective_params =  {
       'l2_leaf_reg': l2_leaf_reg, 
       'min_child_samples': min_child_samples, 
       'max_depth': max_depth, 
       'subsample': subsample, 
       'colsample_bylevel': colsample_bylevel,
       'bagging_temperature': bagging_temperature, 
       'random_strength': random_strength
        }

    
    optuna_CB = KFoldsAverageCBTuning()
    optuna_CB.fit(train_x = X_train, train_y = Y_train, params = objective_params)

    return optuna_CB.rmse

In [88]:
cb_study = optuna.create_study(direction="minimize", pruner = optuna.pruners.SuccessiveHalvingPruner())
cb_study.optimize(objective_CB, n_trials=750, gc_after_trial = True)
print(cb_study.best_trial)

[I 2021-02-07 00:50:41,997] A new study created in memory with name: no-name-cf93c718-aa9f-4b5e-bb7c-b78e3905c3ae


0:	learn: 0.8872644	test: 0.8862255	best: 0.8862255 (0)	total: 10.7ms	remaining: 3m 34s
500:	learn: 0.8872644	test: 0.8862255	best: 0.8862255 (0)	total: 4.66s	remaining: 3m 1s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.8862255494
bestIteration = 0

Shrink model to first 1 iterations.
0:	learn: 0.8861057	test: 0.8908507	best: 0.8908507 (0)	total: 11ms	remaining: 3m 39s
500:	learn: 0.8861057	test: 0.8908507	best: 0.8908507 (0)	total: 4.9s	remaining: 3m 10s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.8908506863
bestIteration = 0

Shrink model to first 1 iterations.
0:	learn: 0.8875962	test: 0.8848945	best: 0.8848945 (0)	total: 11.3ms	remaining: 3m 46s
500:	learn: 0.8875962	test: 0.8848945	best: 0.8848945 (1)	total: 4.83s	remaining: 3m 7s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.8848945453
bestIteration = 1

Shrink model to first 2 iterations.
0:	learn: 0.8864974	test: 0.8892907	best: 0.8892907 (0)	total: 11ms	r

[I 2021-02-07 00:51:14,042] Trial 0 finished with value: 0.8870605767943408 and parameters: {'l2_leaf_reg': 4.722566943946266e-05, 'min_child_samples': 17, 'max_depth': 7, 'subsample': 0.13342, 'colsample_bylevel': 2.9756133929464157e-08, 'bagging_temperature': 137364, 'random_strength': 3}. Best is trial 0 with value: 0.8870605767943408.


0:	learn: 0.8872644	test: 0.8862255	best: 0.8862255 (0)	total: 10.1ms	remaining: 3m 22s
500:	learn: 0.8727241	test: 0.8705868	best: 0.8705868 (460)	total: 5.17s	remaining: 3m 21s
1000:	learn: 0.8660148	test: 0.8642490	best: 0.8642486 (806)	total: 10.1s	remaining: 3m 11s
1500:	learn: 0.8640810	test: 0.8619520	best: 0.8619520 (1366)	total: 15s	remaining: 3m 4s
2000:	learn: 0.8636062	test: 0.8614920	best: 0.8614915 (1830)	total: 19.6s	remaining: 2m 56s
2500:	learn: 0.8624890	test: 0.8602750	best: 0.8602749 (2317)	total: 24.1s	remaining: 2m 48s
3000:	learn: 0.8604572	test: 0.8585532	best: 0.8585532 (2921)	total: 28.6s	remaining: 2m 42s
3500:	learn: 0.8575884	test: 0.8550723	best: 0.8550723 (3487)	total: 33.5s	remaining: 2m 37s
4000:	learn: 0.8568083	test: 0.8546740	best: 0.8546281 (3729)	total: 38.3s	remaining: 2m 33s
4500:	learn: 0.8562668	test: 0.8541131	best: 0.8541131 (4488)	total: 43.2s	remaining: 2m 28s
5000:	learn: 0.8556066	test: 0.8533288	best: 0.8533288 (4991)	total: 48s	remainin

8000:	learn: 0.8531907	test: 0.8491813	best: 0.8491813 (7929)	total: 1m 14s	remaining: 1m 51s
8500:	learn: 0.8531465	test: 0.8491812	best: 0.8491554 (8282)	total: 1m 19s	remaining: 1m 47s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.8491554211
bestIteration = 8282

Shrink model to first 8283 iterations.


...

0:	learn: 0.8872644	test: 0.8862255	best: 0.8862255 (0)	total: 12.6ms	remaining: 4m 12s
500:	learn: 0.8576206	test: 0.8565259	best: 0.8565259 (500)	total: 5.49s	remaining: 3m 33s
1000:	learn: 0.8529765	test: 0.8514443	best: 0.8514443 (987)	total: 11.4s	remaining: 3m 36s
1500:	learn: 0.8516664	test: 0.8500652	best: 0.8500652 (1486)	total: 16.7s	remaining: 3m 26s
2000:	learn: 0.8503593	test: 0.8488429	best: 0.8488429 (1999)	total: 22.2s	remaining: 3m 19s
2500:	learn: 0.8494999	test: 0.8480009	best: 0.8479821 (2479)	total: 27.3s	remaining: 3m 10s
3000:	learn: 0.8482471	test: 0.8469520	best: 0.8469520 (2995)	total: 32.5s	remaining: 3m 3s
3500:	learn: 0.8477579	test: 0.8465172	best: 0.8464934 (3448)	total: 37.6s	remaining: 2m 57s
4000:	learn: 0.8473442	test: 0.8461414	best: 0.8461226 (3938)	total: 42.7s	remaining: 2m 50s
4500:	learn: 0.8467461	test: 0.8457067	best: 0.8457067 (4497)	total: 47.7s	remaining: 2m 44s
5000:	learn: 0.8463973	test: 0.8453769	best: 0.8453769 (4993)	total: 53s	remain

7000:	learn: 0.8438646	test: 0.8501653	best: 0.8501412 (6816)	total: 1m 13s	remaining: 2m 15s
7500:	learn: 0.8435347	test: 0.8500664	best: 0.8500520 (7206)	total: 1m 18s	remaining: 2m 10s
8000:	learn: 0.8433474	test: 0.8499668	best: 0.8499319 (7817)	total: 1m 23s	remaining: 2m 5s
8500:	learn: 0.8431980	test: 0.8498755	best: 0.8498287 (8384)	total: 1m 28s	remaining: 1m 59s
9000:	learn: 0.8430949	test: 0.8497301	best: 0.8497219 (8941)	total: 1m 33s	remaining: 1m 54s
9500:	learn: 0.8429628	test: 0.8496565	best: 0.8496421 (9464)	total: 1m 38s	remaining: 1m 49s
10000:	learn: 0.8428356	test: 0.8495032	best: 0.8495032 (9986)	total: 1m 43s	remaining: 1m 43s
10500:	learn: 0.8426666	test: 0.8494709	best: 0.8494142 (10360)	total: 1m 49s	remaining: 1m 38s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.8494141556
bestIteration = 10360

Shrink model to first 10361 iterations.
0:	learn: 0.8878147	test: 0.8840222	best: 0.8840222 (0)	total: 11.2ms	remaining: 3m 44s
500:	learn: 0.8

[I 2021-02-07 16:08:59,604] Trial 151 finished with value: 0.8465980446623075 and parameters: {'l2_leaf_reg': 8.35590623666593e-10, 'min_child_samples': 25, 'max_depth': 10, 'subsample': 0.6454200000000001, 'colsample_bylevel': 0.003909854106832776, 'bagging_temperature': 151262, 'random_strength': 24}. Best is trial 142 with value: 0.846257690550051.


0:	learn: 0.8872644	test: 0.8862255	best: 0.8862255 (0)	total: 11.9ms	remaining: 3m 58s
500:	learn: 0.8542927	test: 0.8526392	best: 0.8526392 (498)	total: 5.68s	remaining: 3m 40s
1000:	learn: 0.8507159	test: 0.8490430	best: 0.8490430 (998)	total: 11.4s	remaining: 3m 36s
1500:	learn: 0.8494257	test: 0.8479262	best: 0.8479254 (1453)	total: 17.1s	remaining: 3m 30s
2000:	learn: 0.8481870	test: 0.8469857	best: 0.8469804 (1877)	total: 23s	remaining: 3m 26s
2500:	learn: 0.8470191	test: 0.8464856	best: 0.8463793 (2453)	total: 28.7s	remaining: 3m 20s
3000:	learn: 0.8458295	test: 0.8456351	best: 0.8456350 (2988)	total: 34.2s	remaining: 3m 13s
3500:	learn: 0.8453398	test: 0.8455742	best: 0.8455543 (3222)	total: 39.5s	remaining: 3m 6s
4000:	learn: 0.8447494	test: 0.8454619	best: 0.8453828 (3719)	total: 44.9s	remaining: 2m 59s
4500:	learn: 0.8443106	test: 0.8452020	best: 0.8452003 (4489)	total: 50.2s	remaining: 2m 52s
5000:	learn: 0.8438785	test: 0.8450689	best: 0.8450682 (4988)	total: 55.6s	remain

KeyboardInterrupt: 

In [89]:
optuna.visualization.plot_optimization_history(cb_study)

In [90]:
optuna.visualization.plot_parallel_coordinate(cb_study)

In [92]:
optuna.visualization.plot_param_importances(cb_study)

In [95]:
optuna_CB_df = cb_study.trials_dataframe()
optuna_CB_df

,number,value,datetime_start,datetime_complete,duration,params_bagging_temperature,params_colsample_bylevel,params_l2_leaf_reg,params_max_depth,params_min_child_samples,params_random_strength,params_subsample,state
0,0,0.887061,2021-02-07 00:50:41.999119,2021-02-07 00:51:14.041956,0 days 00:00:32.042837,137364,2.975613e-08,4.722567e-05,7,17,3,0.13342,COMPLETE
1,1,0.853776,2021-02-07 00:51:14.274167,2021-02-07 00:57:53.580470,0 days 00:06:39.306303,175659,5.931017e-04,6.644096e-01,16,17,31,0.22963,COMPLETE
2,2,0.846948,2021-02-07 00:57:53.758519,2021-02-07 01:06:52.333984,0 days 00:08:58.575465,143483,3.045545e-03,2.545311e-01,6,17,19,0.71678,COMPLETE
3,3,0.846816,2021-02-07 01:06:52.510025,2021-02-07 01:10:15.278217,0 days 00:03:22.768192,130115,1.080656e-02,1.179812e-05,16,25,33,0.83928,COMPLETE
4,4,0.848262,2021-02-07 01:10:15.454257,2021-02-07 01:11:46.755391,0 days 00:01:31.301134,3727,4.343103e-02,3.687310e-10,4,21,37,0.39621,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,148,0.847163,2021-02-07 15:32:11.571752,2021-02-07 15:45:24.685888,0 days 00:13:13.114136,134643,2.360532e-03,1.601588e-08,12,25,1,0.74600,COMPLETE
149,149,0.846814,2021-02-07 15:45:24.893147,2021-02-07 15:49:39.162572,0 days 00:04:14.269425,137673,7.638210e-03,2.766562e-09,11,27,27,0.78787,COMPLETE
150,150,0.846537,2021-02-07 15:49:39.341122,2021-02-07 15:59:21.124480,0 days 00:09:41.783358,149031,3.918070e-03,9.156790e-10,10,25,25,0.64968,COMPLETE
151,151,0.846598,2021-02-07 15:59:21.322524,2021-02-07 16:08:59.604255,0 days 00:09:38.281731,151262,3.909854e-03,8.355906e-10,10,25,24,0.64542,COMPLETE


# Manual Hyperparameter Search

Low Learning Rate is extremely important on how well the model will train but it will come at the cost of the time of training. Therefore, I have taken the top 5 performing parameters from the first stage search and will now use a low learning rate to see which performs the best.

In [118]:
top5CBparams = optuna_CB_df.sort_values('value', ascending = True).iloc[:, 5:11].head(5)
top5CBparams

,params_bagging_temperature,params_colsample_bylevel,params_l2_leaf_reg,params_max_depth,params_min_child_samples,params_random_strength
142,139023,0.007034,4.342317e-09,11,27,24
102,129611,0.006326,8.374335e-09,14,27,19
51,184787,0.005349,7.302161e-10,13,9,29
71,138269,0.008023,5.135046e-09,9,21,29
103,131041,0.006345,1.707945e-09,14,29,20


In [120]:
top5CBparams.columns = ['bagging_temperature',  
                        'colsample_bylevel',
                        'l2_leaf_reg',
                        'max_depth',
                        'min_child_samples',
                        'random_strength']

In [130]:
top5CBparams = top5CBparams.to_dict('records')

In [186]:
class KFoldCBFineTuning():
    def __init__(self):
        self.models = []
        self.kfolds = KFold(n_splits = 5, shuffle = False)
        
        
    def fit(self, train_x, train_y, params):
        oof_preds = np.zeros_like(train_y)
        self.train_x = train_x.values
        self.train_y = train_y.values
        
        for train_idx, val_idx in self.kfolds.split(train_x):
            X_trainCV, X_valCV = self.train_x[train_idx], self.train_x[val_idx]
            Y_trainCV, Y_valCV = self.train_y[train_idx], self.train_y[val_idx]
            
            model = cb.CatBoostRegressor(
                        random_state = 50,
                        loss_function = 'RMSE',
                        silent = True,
                        learning_rate = 0.01,
                        iterations = 1000000,
                        cat_features = list(range(10)),
                        **params
                        )
            
            model.fit(X_trainCV, Y_trainCV, 
                      early_stopping_rounds=1000,
                      eval_set=[(X_valCV, Y_valCV)], verbose_eval=5000)
            
            self.models.append(model)
            oof_pred = model.predict(X_valCV)
            oof_preds[val_idx] = oof_pred
            
        self.oof_preds = oof_preds
        
        self.rmse = mean_squared_error(Y_train, oof_preds, squared = False)

    def predict(self, test_x):
        preds = []
        for model in tqdm.tqdm(self.models):
            pred = model.predict(test_x)
            preds.append(pred)
        preds = np.mean(preds, axis=0)
        return preds

In [153]:
top5CBparams[0]

{'bagging_temperature': 139023,
 'colsample_bylevel': 0.007034392937411116,
 'l2_leaf_reg': 4.342316856215848e-09,
 'max_depth': 11,
 'min_child_samples': 27,
 'random_strength': 24}

In [154]:
# manually find the best hyperparameter from top 5 with low learning rate 
top1CBtrained = KFoldCBFineTuning()
top1CBtrained.fit(train_x = X_train, train_y = Y_train, params = top5CBparams[0])

0:	learn: 0.8872527	test: 0.8862132	best: 0.8862132 (0)	total: 17ms	remaining: 4h 43m 29s
5000:	learn: 0.8707767	test: 0.8694672	best: 0.8694672 (4999)	total: 1m 11s	remaining: 3h 57m 9s
10000:	learn: 0.8631920	test: 0.8616752	best: 0.8616752 (9995)	total: 2m 16s	remaining: 3h 45m 14s
15000:	learn: 0.8595370	test: 0.8578657	best: 0.8578657 (14994)	total: 3m 20s	remaining: 3h 39m 5s
20000:	learn: 0.8573543	test: 0.8555522	best: 0.8555522 (20000)	total: 4m 24s	remaining: 3h 36m 11s
25000:	learn: 0.8558316	test: 0.8539496	best: 0.8539496 (24987)	total: 5m 28s	remaining: 3h 33m 48s
30000:	learn: 0.8547033	test: 0.8527556	best: 0.8527556 (29994)	total: 6m 34s	remaining: 3h 32m 50s
35000:	learn: 0.8538300	test: 0.8518503	best: 0.8518503 (34993)	total: 7m 40s	remaining: 3h 31m 37s
40000:	learn: 0.8531668	test: 0.8511642	best: 0.8511642 (39969)	total: 8m 46s	remaining: 3h 30m 25s
45000:	learn: 0.8526032	test: 0.8506038	best: 0.8506038 (44967)	total: 9m 53s	remaining: 3h 29m 47s
50000:	learn: 0

405000:	learn: 0.8426506	test: 0.8428986	best: 0.8428986 (404993)	total: 1h 28m 49s	remaining: 2h 10m 30s
410000:	learn: 0.8425892	test: 0.8428716	best: 0.8428716 (409997)	total: 1h 29m 55s	remaining: 2h 9m 23s
415000:	learn: 0.8425405	test: 0.8428511	best: 0.8428510 (414973)	total: 1h 31m	remaining: 2h 8m 17s
420000:	learn: 0.8424886	test: 0.8428187	best: 0.8428187 (419989)	total: 1h 32m 5s	remaining: 2h 7m 10s
425000:	learn: 0.8424343	test: 0.8427954	best: 0.8427954 (424995)	total: 1h 33m 11s	remaining: 2h 6m 4s
430000:	learn: 0.8423781	test: 0.8427670	best: 0.8427668 (429734)	total: 1h 34m 15s	remaining: 2h 4m 57s
435000:	learn: 0.8423214	test: 0.8427358	best: 0.8427358 (434998)	total: 1h 35m 21s	remaining: 2h 3m 51s
440000:	learn: 0.8422705	test: 0.8427073	best: 0.8427071 (439917)	total: 1h 36m 26s	remaining: 2h 2m 45s
445000:	learn: 0.8422230	test: 0.8426866	best: 0.8426866 (444958)	total: 1h 37m 31s	remaining: 2h 1m 38s
450000:	learn: 0.8421677	test: 0.8426594	best: 0.8426594 (44

235000:	learn: 0.8431351	test: 0.8518406	best: 0.8518406 (234995)	total: 52m 6s	remaining: 2h 49m 38s
240000:	learn: 0.8430445	test: 0.8517899	best: 0.8517892 (239748)	total: 53m 16s	remaining: 2h 48m 43s
245000:	learn: 0.8429455	test: 0.8517292	best: 0.8517290 (244980)	total: 54m 29s	remaining: 2h 47m 55s
250000:	learn: 0.8428631	test: 0.8516844	best: 0.8516844 (249997)	total: 55m 39s	remaining: 2h 46m 57s
255000:	learn: 0.8427712	test: 0.8516378	best: 0.8516373 (254830)	total: 56m 45s	remaining: 2h 45m 50s
260000:	learn: 0.8426857	test: 0.8515810	best: 0.8515808 (259986)	total: 57m 55s	remaining: 2h 44m 50s
265000:	learn: 0.8425822	test: 0.8515264	best: 0.8515264 (264969)	total: 59m 6s	remaining: 2h 43m 56s
270000:	learn: 0.8425011	test: 0.8514704	best: 0.8514704 (269995)	total: 1h 16s	remaining: 2h 42m 59s
275000:	learn: 0.8424143	test: 0.8514293	best: 0.8514293 (274997)	total: 1h 1m 27s	remaining: 2h 42m 2s
280000:	learn: 0.8423306	test: 0.8513845	best: 0.8513844 (279977)	total: 1h

630000:	learn: 0.8385128	test: 0.8496391	best: 0.8496390 (629683)	total: 2h 16m 32s	remaining: 1h 20m 11s
635000:	learn: 0.8384780	test: 0.8496210	best: 0.8496210 (634992)	total: 2h 17m 39s	remaining: 1h 19m 7s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 0.8496118498
bestIteration = 638353

Shrink model to first 638354 iterations.
0:	learn: 0.8875943	test: 0.8848933	best: 0.8848933 (0)	total: 15.5ms	remaining: 4h 18m 48s
5000:	learn: 0.8708938	test: 0.8687535	best: 0.8687535 (4997)	total: 1m 3s	remaining: 3h 31m 14s
10000:	learn: 0.8630972	test: 0.8612829	best: 0.8612829 (9999)	total: 2m 9s	remaining: 3h 33m 2s
15000:	learn: 0.8593557	test: 0.8578373	best: 0.8578373 (14998)	total: 3m 13s	remaining: 3h 32m 4s
20000:	learn: 0.8571343	test: 0.8558578	best: 0.8558578 (19996)	total: 4m 18s	remaining: 3h 31m 17s
25000:	learn: 0.8555632	test: 0.8544610	best: 0.8544610 (24995)	total: 5m 28s	remaining: 3h 33m 24s
30000:	learn: 0.8544013	test: 0.8534264	best: 0.8534264 (2

385000:	learn: 0.8423690	test: 0.8446897	best: 0.8446897 (384984)	total: 1h 19m 2s	remaining: 2h 6m 15s
390000:	learn: 0.8423077	test: 0.8446586	best: 0.8446586 (389964)	total: 1h 20m 4s	remaining: 2h 5m 14s
395000:	learn: 0.8422561	test: 0.8446334	best: 0.8446334 (395000)	total: 1h 21m 6s	remaining: 2h 4m 13s
400000:	learn: 0.8421951	test: 0.8445990	best: 0.8445990 (399998)	total: 1h 22m 8s	remaining: 2h 3m 12s
405000:	learn: 0.8421433	test: 0.8445767	best: 0.8445766 (404937)	total: 1h 23m 10s	remaining: 2h 2m 12s
410000:	learn: 0.8420852	test: 0.8445493	best: 0.8445493 (409996)	total: 1h 24m 13s	remaining: 2h 1m 11s
415000:	learn: 0.8420327	test: 0.8445230	best: 0.8445230 (414981)	total: 1h 25m 16s	remaining: 2h 11s
420000:	learn: 0.8419816	test: 0.8445034	best: 0.8445033 (419978)	total: 1h 26m 18s	remaining: 1h 59m 10s
425000:	learn: 0.8419274	test: 0.8444761	best: 0.8444761 (424992)	total: 1h 27m 21s	remaining: 1h 58m 10s
430000:	learn: 0.8418728	test: 0.8444539	best: 0.8444538 (42

100000:	learn: 0.8475640	test: 0.8524079	best: 0.8524079 (99994)	total: 20m 57s	remaining: 3h 8m 34s
105000:	learn: 0.8473515	test: 0.8522353	best: 0.8522353 (104995)	total: 21m 56s	remaining: 3h 7m
110000:	learn: 0.8471417	test: 0.8520718	best: 0.8520718 (109997)	total: 22m 55s	remaining: 3h 5m 27s
115000:	learn: 0.8469253	test: 0.8519076	best: 0.8519076 (114988)	total: 23m 54s	remaining: 3h 3m 59s
120000:	learn: 0.8467399	test: 0.8517645	best: 0.8517645 (119998)	total: 24m 53s	remaining: 3h 2m 33s
125000:	learn: 0.8465331	test: 0.8516070	best: 0.8516070 (124989)	total: 25m 52s	remaining: 3h 1m 8s
130000:	learn: 0.8463420	test: 0.8514709	best: 0.8514709 (129999)	total: 26m 51s	remaining: 2h 59m 42s
135000:	learn: 0.8461778	test: 0.8513563	best: 0.8513563 (134997)	total: 27m 50s	remaining: 2h 58m 25s
140000:	learn: 0.8460201	test: 0.8512264	best: 0.8512262 (139992)	total: 28m 49s	remaining: 2h 57m 2s
145000:	learn: 0.8458728	test: 0.8511110	best: 0.8511110 (144990)	total: 29m 48s	remai

500000:	learn: 0.8401537	test: 0.8477399	best: 0.8477395 (499659)	total: 1h 43m 3s	remaining: 1h 43m 3s
505000:	learn: 0.8401086	test: 0.8477224	best: 0.8477224 (504999)	total: 1h 44m 9s	remaining: 1h 42m 6s
510000:	learn: 0.8400643	test: 0.8477016	best: 0.8477015 (509986)	total: 1h 45m 16s	remaining: 1h 41m 8s
515000:	learn: 0.8400249	test: 0.8476870	best: 0.8476867 (514430)	total: 1h 46m 22s	remaining: 1h 40m 10s
520000:	learn: 0.8399781	test: 0.8476637	best: 0.8476635 (519990)	total: 1h 47m 28s	remaining: 1h 39m 12s
525000:	learn: 0.8399323	test: 0.8476442	best: 0.8476442 (524996)	total: 1h 48m 35s	remaining: 1h 38m 14s
530000:	learn: 0.8398876	test: 0.8476267	best: 0.8476267 (529994)	total: 1h 49m 42s	remaining: 1h 37m 17s
535000:	learn: 0.8398394	test: 0.8476116	best: 0.8476116 (534982)	total: 1h 50m 49s	remaining: 1h 36m 19s
540000:	learn: 0.8397914	test: 0.8475920	best: 0.8475920 (539998)	total: 1h 51m 56s	remaining: 1h 35m 21s
545000:	learn: 0.8397427	test: 0.8475696	best: 0.84

225000:	learn: 0.8455085	test: 0.8429172	best: 0.8429171 (224980)	total: 51m 55s	remaining: 2h 58m 52s
230000:	learn: 0.8453949	test: 0.8428411	best: 0.8428411 (229999)	total: 53m 7s	remaining: 2h 57m 51s
235000:	learn: 0.8452982	test: 0.8427720	best: 0.8427719 (234973)	total: 54m 19s	remaining: 2h 56m 51s
240000:	learn: 0.8452088	test: 0.8427090	best: 0.8427090 (239995)	total: 55m 31s	remaining: 2h 55m 49s
245000:	learn: 0.8451180	test: 0.8426472	best: 0.8426472 (244995)	total: 56m 43s	remaining: 2h 54m 47s
250000:	learn: 0.8450332	test: 0.8425863	best: 0.8425863 (249994)	total: 57m 55s	remaining: 2h 53m 46s
255000:	learn: 0.8449462	test: 0.8425235	best: 0.8425233 (254976)	total: 59m 10s	remaining: 2h 52m 53s
260000:	learn: 0.8448633	test: 0.8424620	best: 0.8424620 (259999)	total: 1h 24s	remaining: 2h 51m 55s
265000:	learn: 0.8447612	test: 0.8424000	best: 0.8424000 (264990)	total: 1h 1m 35s	remaining: 2h 50m 48s
270000:	learn: 0.8446779	test: 0.8423479	best: 0.8423478 (269919)	total: 

In [187]:
mean_squared_error(Y_train, top1CBtrained.oof_preds, squared = False)

0.8446764112679253

In [188]:
mean_squared_error(y_true = Y_val, 
                   y_pred = top1CBtrained.predict(X_val),
                   squared = False)                   

TypeError: 'module' object is not callable

In [145]:
top2CBtrained = KFoldCBFineTuning()
top2CBtrained.fit(train_x = X_train, train_y = Y_train, params = top5CBparams[1])

0:	learn: 0.8872644	test: 0.8862255	best: 0.8862255 (0)	total: 11.2ms	remaining: 1h 33m 31s
1000:	learn: 0.8846043	test: 0.8835386	best: 0.8835386 (1000)	total: 10.5s	remaining: 1h 27m 24s
2000:	learn: 0.8825083	test: 0.8813712	best: 0.8813712 (1990)	total: 21.1s	remaining: 1h 27m 33s
3000:	learn: 0.8790820	test: 0.8779080	best: 0.8779080 (3000)	total: 31.8s	remaining: 1h 27m 39s
4000:	learn: 0.8767086	test: 0.8755301	best: 0.8755301 (3987)	total: 42.1s	remaining: 1h 27m 1s
5000:	learn: 0.8748365	test: 0.8736093	best: 0.8736093 (4991)	total: 52.7s	remaining: 1h 26m 59s
6000:	learn: 0.8729260	test: 0.8716614	best: 0.8716614 (5992)	total: 1m 2s	remaining: 1h 26m 20s
7000:	learn: 0.8713607	test: 0.8700598	best: 0.8700598 (7000)	total: 1m 14s	remaining: 1h 26m 51s
8000:	learn: 0.8697337	test: 0.8684114	best: 0.8684114 (7979)	total: 1m 24s	remaining: 1h 27m 2s
9000:	learn: 0.8686045	test: 0.8672800	best: 0.8672800 (8997)	total: 1m 35s	remaining: 1h 26m 46s
10000:	learn: 0.8673711	test: 0.86

83000:	learn: 0.8516843	test: 0.8497109	best: 0.8497109 (82999)	total: 14m 33s	remaining: 1h 13m 8s
84000:	learn: 0.8516115	test: 0.8496341	best: 0.8496341 (83989)	total: 14m 44s	remaining: 1h 12m 59s
85000:	learn: 0.8515550	test: 0.8495811	best: 0.8495811 (84989)	total: 14m 54s	remaining: 1h 12m 48s
86000:	learn: 0.8515001	test: 0.8495320	best: 0.8495320 (85993)	total: 15m 5s	remaining: 1h 12m 37s
87000:	learn: 0.8514539	test: 0.8494884	best: 0.8494884 (87000)	total: 15m 15s	remaining: 1h 12m 26s
88000:	learn: 0.8513977	test: 0.8494360	best: 0.8494359 (87946)	total: 15m 25s	remaining: 1h 12m 14s
89000:	learn: 0.8513474	test: 0.8493953	best: 0.8493953 (88998)	total: 15m 35s	remaining: 1h 12m 1s
90000:	learn: 0.8512982	test: 0.8493483	best: 0.8493483 (89998)	total: 15m 45s	remaining: 1h 11m 48s
91000:	learn: 0.8512450	test: 0.8493006	best: 0.8493006 (90986)	total: 15m 56s	remaining: 1h 11m 39s
92000:	learn: 0.8511927	test: 0.8492539	best: 0.8492539 (91999)	total: 16m 7s	remaining: 1h 11

165000:	learn: 0.8485801	test: 0.8469589	best: 0.8469589 (164998)	total: 28m 25s	remaining: 57m 41s
166000:	learn: 0.8485452	test: 0.8469318	best: 0.8469317 (165969)	total: 28m 35s	remaining: 57m 31s
167000:	learn: 0.8485115	test: 0.8469009	best: 0.8469009 (166996)	total: 28m 45s	remaining: 57m 21s
168000:	learn: 0.8484840	test: 0.8468781	best: 0.8468781 (167998)	total: 28m 55s	remaining: 57m 10s
169000:	learn: 0.8484613	test: 0.8468544	best: 0.8468544 (168998)	total: 29m 6s	remaining: 56m 59s
170000:	learn: 0.8484418	test: 0.8468346	best: 0.8468346 (169994)	total: 29m 16s	remaining: 56m 48s
171000:	learn: 0.8484102	test: 0.8468095	best: 0.8468094 (170982)	total: 29m 26s	remaining: 56m 38s
172000:	learn: 0.8483824	test: 0.8467879	best: 0.8467879 (171968)	total: 29m 36s	remaining: 56m 28s
173000:	learn: 0.8483589	test: 0.8467663	best: 0.8467663 (172984)	total: 29m 47s	remaining: 56m 18s
174000:	learn: 0.8483308	test: 0.8467384	best: 0.8467384 (173973)	total: 29m 57s	remaining: 56m 7s
17

248000:	learn: 0.8467779	test: 0.8454931	best: 0.8454930 (247973)	total: 42m 37s	remaining: 43m 18s
249000:	learn: 0.8467594	test: 0.8454794	best: 0.8454793 (248975)	total: 42m 47s	remaining: 43m 8s
250000:	learn: 0.8467430	test: 0.8454674	best: 0.8454671 (249974)	total: 42m 57s	remaining: 42m 57s
251000:	learn: 0.8467249	test: 0.8454490	best: 0.8454490 (250988)	total: 43m 7s	remaining: 42m 47s
252000:	learn: 0.8467130	test: 0.8454402	best: 0.8454401 (251843)	total: 43m 18s	remaining: 42m 36s
253000:	learn: 0.8466976	test: 0.8454294	best: 0.8454290 (252880)	total: 43m 28s	remaining: 42m 26s
254000:	learn: 0.8466832	test: 0.8454171	best: 0.8454170 (253989)	total: 43m 38s	remaining: 42m 16s
255000:	learn: 0.8466657	test: 0.8454016	best: 0.8454014 (254991)	total: 43m 49s	remaining: 42m 6s
256000:	learn: 0.8466469	test: 0.8453899	best: 0.8453899 (255942)	total: 43m 59s	remaining: 41m 55s
257000:	learn: 0.8466356	test: 0.8453795	best: 0.8453795 (256991)	total: 44m 9s	remaining: 41m 45s
2580

331000:	learn: 0.8454983	test: 0.8445193	best: 0.8445193 (330995)	total: 56m 53s	remaining: 29m 2s
332000:	learn: 0.8454890	test: 0.8445114	best: 0.8445114 (331992)	total: 57m 3s	remaining: 28m 52s
333000:	learn: 0.8454756	test: 0.8445033	best: 0.8445032 (332971)	total: 57m 13s	remaining: 28m 41s
334000:	learn: 0.8454616	test: 0.8444966	best: 0.8444966 (333994)	total: 57m 23s	remaining: 28m 31s
335000:	learn: 0.8454485	test: 0.8444874	best: 0.8444874 (334999)	total: 57m 34s	remaining: 28m 21s
336000:	learn: 0.8454314	test: 0.8444756	best: 0.8444756 (335995)	total: 57m 44s	remaining: 28m 11s
337000:	learn: 0.8454228	test: 0.8444683	best: 0.8444683 (336995)	total: 57m 54s	remaining: 28m
338000:	learn: 0.8454101	test: 0.8444618	best: 0.8444615 (337979)	total: 58m 5s	remaining: 27m 50s
339000:	learn: 0.8453958	test: 0.8444508	best: 0.8444507 (338981)	total: 58m 16s	remaining: 27m 40s
340000:	learn: 0.8453799	test: 0.8444371	best: 0.8444371 (339982)	total: 58m 26s	remaining: 27m 30s
341000:

...

133000:	learn: 0.8497873	test: 0.8463333	best: 0.8463330 (132962)	total: 22m 34s	remaining: 1h 2m 18s
134000:	learn: 0.8497433	test: 0.8462967	best: 0.8462967 (133997)	total: 22m 44s	remaining: 1h 2m 7s
135000:	learn: 0.8497175	test: 0.8462773	best: 0.8462773 (134994)	total: 22m 54s	remaining: 1h 1m 57s
136000:	learn: 0.8496860	test: 0.8462465	best: 0.8462464 (135948)	total: 23m 4s	remaining: 1h 1m 46s
137000:	learn: 0.8496565	test: 0.8462154	best: 0.8462154 (136990)	total: 23m 14s	remaining: 1h 1m 36s
138000:	learn: 0.8496221	test: 0.8461850	best: 0.8461850 (137968)	total: 23m 25s	remaining: 1h 1m 25s
139000:	learn: 0.8495929	test: 0.8461559	best: 0.8461559 (138981)	total: 23m 34s	remaining: 1h 1m 14s
140000:	learn: 0.8495685	test: 0.8461372	best: 0.8461372 (139978)	total: 23m 45s	remaining: 1h 1m 4s
141000:	learn: 0.8495373	test: 0.8461104	best: 0.8461101 (140956)	total: 23m 55s	remaining: 1h 53s
142000:	learn: 0.8495025	test: 0.8460809	best: 0.8460809 (141996)	total: 24m 5s	remainin

216000:	learn: 0.8476308	test: 0.8444705	best: 0.8444705 (215993)	total: 36m 35s	remaining: 48m 7s
217000:	learn: 0.8476073	test: 0.8444512	best: 0.8444512 (216991)	total: 36m 45s	remaining: 47m 56s
218000:	learn: 0.8475886	test: 0.8444376	best: 0.8444374 (217907)	total: 36m 56s	remaining: 47m 46s
219000:	learn: 0.8475687	test: 0.8444209	best: 0.8444203 (218921)	total: 37m 6s	remaining: 47m 36s
220000:	learn: 0.8475496	test: 0.8444047	best: 0.8444046 (219979)	total: 37m 16s	remaining: 47m 26s
221000:	learn: 0.8475257	test: 0.8443866	best: 0.8443866 (220998)	total: 37m 26s	remaining: 47m 16s
222000:	learn: 0.8475073	test: 0.8443746	best: 0.8443744 (221986)	total: 37m 36s	remaining: 47m 5s
223000:	learn: 0.8474954	test: 0.8443633	best: 0.8443633 (222998)	total: 37m 46s	remaining: 46m 55s
224000:	learn: 0.8474775	test: 0.8443453	best: 0.8443453 (223993)	total: 37m 56s	remaining: 46m 45s
225000:	learn: 0.8474621	test: 0.8443315	best: 0.8443315 (225000)	total: 38m 6s	remaining: 46m 34s
2260

299000:	learn: 0.8461935	test: 0.8433422	best: 0.8433422 (298990)	total: 50m 37s	remaining: 34m 1s
300000:	learn: 0.8461789	test: 0.8433299	best: 0.8433299 (299992)	total: 50m 47s	remaining: 33m 51s
301000:	learn: 0.8461676	test: 0.8433216	best: 0.8433215 (300965)	total: 50m 57s	remaining: 33m 41s
302000:	learn: 0.8461547	test: 0.8433111	best: 0.8433111 (301995)	total: 51m 7s	remaining: 33m 30s
303000:	learn: 0.8461398	test: 0.8433001	best: 0.8433000 (302958)	total: 51m 17s	remaining: 33m 20s
304000:	learn: 0.8461269	test: 0.8432926	best: 0.8432926 (303969)	total: 51m 27s	remaining: 33m 10s
305000:	learn: 0.8461078	test: 0.8432750	best: 0.8432749 (304970)	total: 51m 37s	remaining: 33m
306000:	learn: 0.8460901	test: 0.8432646	best: 0.8432646 (306000)	total: 51m 47s	remaining: 32m 50s
307000:	learn: 0.8460706	test: 0.8432514	best: 0.8432514 (306995)	total: 51m 57s	remaining: 32m 40s
308000:	learn: 0.8460584	test: 0.8432409	best: 0.8432409 (307971)	total: 52m 8s	remaining: 32m 30s
309000:

381000:	learn: 0.8451509	test: 0.8425944	best: 0.8425944 (380996)	total: 1h 4m 26s	remaining: 20m 7s
382000:	learn: 0.8451414	test: 0.8425898	best: 0.8425898 (381976)	total: 1h 4m 36s	remaining: 19m 57s
383000:	learn: 0.8451294	test: 0.8425824	best: 0.8425824 (382980)	total: 1h 4m 47s	remaining: 19m 47s
384000:	learn: 0.8451150	test: 0.8425738	best: 0.8425738 (383958)	total: 1h 4m 57s	remaining: 19m 37s
385000:	learn: 0.8451040	test: 0.8425672	best: 0.8425667 (384970)	total: 1h 5m 7s	remaining: 19m 27s
386000:	learn: 0.8450956	test: 0.8425612	best: 0.8425611 (385975)	total: 1h 5m 17s	remaining: 19m 16s
387000:	learn: 0.8450846	test: 0.8425541	best: 0.8425541 (386974)	total: 1h 5m 27s	remaining: 19m 6s
388000:	learn: 0.8450742	test: 0.8425464	best: 0.8425464 (387993)	total: 1h 5m 37s	remaining: 18m 56s
389000:	learn: 0.8450656	test: 0.8425405	best: 0.8425404 (388968)	total: 1h 5m 47s	remaining: 18m 46s
390000:	learn: 0.8450556	test: 0.8425337	best: 0.8425337 (389999)	total: 1h 5m 57s	re

462000:	learn: 0.8443098	test: 0.8420518	best: 0.8420518 (461985)	total: 1h 18m 5s	remaining: 6m 25s
463000:	learn: 0.8442995	test: 0.8420459	best: 0.8420458 (462993)	total: 1h 18m 15s	remaining: 6m 15s
464000:	learn: 0.8442899	test: 0.8420393	best: 0.8420393 (463991)	total: 1h 18m 25s	remaining: 6m 5s
465000:	learn: 0.8442816	test: 0.8420354	best: 0.8420354 (464996)	total: 1h 18m 35s	remaining: 5m 54s
466000:	learn: 0.8442712	test: 0.8420282	best: 0.8420281 (465916)	total: 1h 18m 45s	remaining: 5m 44s
467000:	learn: 0.8442599	test: 0.8420202	best: 0.8420202 (466997)	total: 1h 18m 55s	remaining: 5m 34s
468000:	learn: 0.8442503	test: 0.8420148	best: 0.8420148 (467976)	total: 1h 19m 5s	remaining: 5m 24s
469000:	learn: 0.8442436	test: 0.8420105	best: 0.8420103 (468941)	total: 1h 19m 15s	remaining: 5m 14s
470000:	learn: 0.8442364	test: 0.8420038	best: 0.8420038 (470000)	total: 1h 19m 25s	remaining: 5m 4s
471000:	learn: 0.8442288	test: 0.8419972	best: 0.8419972 (470985)	total: 1h 19m 36s	re

In [ ]:
top3CBtrained = KFoldCBFineTuning()
top3CBtrained.fit(train_x = X_train, train_y = Y_train, params = top5CBparams[2])

In [ ]:
top4CBtrained = KFoldCBFineTuning()
top4CBtrained.fit(train_x = X_train, train_y = Y_train, params = top5CBparams[3])

In [ ]:
top5CBtrained = KFoldCBFineTuning()
top5CBtrained.fit(train_x = X_train, train_y = Y_train, params = top5CBparams[4])